In [16]:
%reset -f

In [17]:
import numpy as np

In [18]:
# import numdifftools as nd
# from gwbench.basic_constants import time_fac, strain_fac
# import gwbench.detector_response_derivatives as drd
# import gwbench.snr as snr_mod
# import wf_derivatives_num as wfd_num
# import gwbench.err_deriv_handling as edh

In [19]:
PI = np.pi

In [20]:
import gwbench.basic_constants as bc

In [21]:
# Injection parameter calculation

# e0 = 0.0
# chi1z = 0.0
# chi2z = 0.0

tc = 0.0
phic = 0.0

M1_solar = 1.4    # Change the mass of the binary
M2_solar = 1.4

M_solar = M1_solar + M2_solar          # Total mass in solar mass unit
M_SI = M_solar*bc.Msun                 # Total mass in SI unit
M_sec = M_SI*bc.GNewton/bc.cLight**3   # Total mass in seconds

eta = (M1_solar*M2_solar/(M_solar)**2) 

Mc_solar = M_solar*eta**(3/5)          # Chirp mass in solar mass unit
Mc = M_sec*eta**(3/5)                  # Chirp mass in seconds

# print(Mc_solar)
# print(Mc)

In [22]:
import aiss_model_np as aiss_np     # loading the PSD and can be change from the script
from scipy.integrate import quad    # 

In [23]:
# Choose the PSD and the desired frequency range

# Fs = 4096


fs = 20
flso = (6.**(3/2) * PI * M_sec)**(-1)

# deltaF = 2**(-2.9)
# f = np.arange(0, Fs, deltaF)

# i_fs = int((fs-0)/deltaF)
# i_flso = int((flso-0)/deltaF) + 1

# f = f[i_fs:i_flso]

f = np.arange(fs, flso, 1)

psd_func = aiss_np.Sh_aLIGO 
psd = psd_func(f)

# Calculating Amplitude corresponding to SNR 10
rho0  = 10

def integrand(f1):
    return f1**(-7/3) / psd_func(f1)

ans, err = quad(integrand, f[0], flso) # quad returns the answer of the quadrature sum and the error, the error is quite high
A =  (rho0**2 / (4*ans))**0.5

DL = ((5./24.)**0.5/PI**(2./3.))*(Mc**(5./6.)/A) # From AISS
DL_Mpc = DL*bc.cLight/bc.Mpc


# print(DL)
# print(A)
# print(flso)
# print(f[-1])
# print(f)

## User Choices

#### choose the desired detectors

In [24]:
# network_spec = ['aLIGO_H']

In [25]:
from network_check import Network # "Network" is a class
import gwbench.wf_class as wfc   # "wfc" is a class

#### initialize the network with the desired detectors

In [ ]:
net = Network()  # "net" is an object inside the class "Network"
net.wf = wfc.Waveform()

# print(net.wf)

#### choose the desired waveform 

In [ ]:
import wf_models.tf2_2_np as tf2_2_np
import wf_models.tf2_2_sp as tf2_2_sp

#### pass the chosen waveform to the network for initialization

In [ ]:
def select_wf_model_quants(self):
    np_mod = tf2_2_np
    sp_mod = tf2_2_sp 
    
    if sp_mod is None: sp_tmp = None
    else:              sp_tmp = sp_mod.hfpc
        
    return np_mod.wf_symbs_string, np_mod.hfpc, sp_tmp

net.wf.wf_symbs_string, net.wf.hfpc_np, net.wf.hfpc_sp = select_wf_model_quants(net)

#### set the injection parameters

In [ ]:
inj_params = {
    'Mc':    Mc_solar,
    'eta':   0.2499,
    'chi1z': 0,
    'chi2z': 0,
    'DL':    DL_Mpc,
    'tc':    tc,
    'phic':  phic,
    'iota':  0,
    'ra':    0,
    'dec':   0.0,
    'psi':   0,
    'gmst0': 0
    }

#### assign with respect to which parameters to take derivatives

In [ ]:
deriv_symbs_string = 'Mc eta tc phic'

#### assign which parameters to convert to cos or log versions

In [ ]:
conv_cos = ()
conv_log = ('Mc', 'eta')

#### choose whether to take Earth's rotation into account

In [ ]:
use_rot = 0

#### pass all these variables to the network

In [ ]:
net.set_net_vars(
    f=f, inj_params=inj_params,
    deriv_symbs_string=deriv_symbs_string,
    conv_cos=conv_cos, conv_log=conv_log,
    use_rot=use_rot
    )

## GW benchmarking

#### compute the WF polarizations and their derivatives

In [ ]:
net.calc_wf_polarizations()

In [ ]:
net.calc_wf_polarizations_derivs_num()

In [ ]:
import gwbench.basic_functions as bfs
import fisher_analysis_tools as fat

In [ ]:
deriv_symbs_list = deriv_symbs_string.split(' ')
deriv_hfp_list = ['del_' + ('log_' + item if item in conv_log else item) + '_hfp' for item in deriv_symbs_list]
del_vs_f_dic = bfs.get_sub_dict(net.del_hfpc,deriv_hfp_list,1)

net.fisher, net.cov, net.wc_fisher, net.cond_num = fat.calc_fisher_cov_matrices(list(del_vs_f_dic.values()), psd, f, 0)
net.errs = fat.get_errs_from_cov(net.cov, net.deriv_variables)


In [ ]:
#print error values

from math import floor, log10

def round_n(x, n):
    return round(x, n - int(floor(log10(abs(x)))) - 1)


# print the contents of the network objects

print("tc(ms): ", round_n(net.errs['tc']*1000,5)) # 1000 here is for sec to msec conversion
print("phic: ", round_n(net.errs['phic'], 5))
print("log_Mch: ", round_n(net.errs['log_Mc']*100,5)) # 100 here is for percentage error
print("log_eta: ", round_n(net.errs['log_eta']*100,5)) # 100 here is for percentage error
# print("log_e0: ", round_n(net.errs['log_e0'],4))
print()


In [ ]:
# tc(ms):  0.51937
# phic:  1.2794
# log_Mch:  0.013327
# log_eta:  1.319

#### setup antenna patterns, location phase factors, and PSDs

In [ ]:
# from numpy import power, logical_and

In [ ]:
# def f1(f):
#     x = f/215
#     return 1.e-49 * (power(x,-4.14) - 5 * power(x, -2) +
#                 111 * (1 - power(x, 2) + power(x, 4) / 2) / (1 + power(x, 2) / 2)) 

In [ ]:
# def f2(f):
#     x = f/245.4
#     return 1.e-48 * ( 0.0152 * power(x,-4.) + 0.2935 * power(x,9./4) +
#                 2.7951 * power(x,3./2) - 6.5080 * power(x,3./4) + 17.7622)

In [ ]:
# f_lo = 10
# f_hi = 2048

for i in range(3):
    net.detectors[i].psd, net.f = f1(f[logical_and(f>=f_lo,f<=f_hi)]), f[logical_and(f>=f_lo,f<=f_hi)]

#### compute the detector responses and their derivatives

net.calc_det_responses()

net.calc_det_responses_derivs_num()

net.calc_snrs()

#### calculate the network and detector Fisher matrices, condition numbers, covariance matrices, error estimates, and inversion errors

net.calc_errors()

net.cov

net.errs

net.print_detectors()

net.snr

def calc_det_responses(self, wf, inj_params, Fp, Fc, Flp):
        hfp, hfc = wf.eval_np_func(self.f,bfs.get_sub_dict(inj_params,wf.wf_symbs_string))
        return Flp * (hfp * Fp + hfc * Fc)

hf1 = calc_det_responses(net, net.wf, inj_params, Fp, Fc, Flp)

net.hfp

In [ ]:
# psd1 = f1(f[logical_and(f>=f_lo,f<=f_hi)])
# cond_sup = 1e15
# by_element = 0

In [ ]:
# def get_conv_del_eval_dic(del_eval_dic, params_dic, conv_cos, conv_log, deriv_symbs_string):
#     if conv_cos is None and conv_log is None:
#         return del_eval_dic, {}
#     else:
#         conv_dic = {}
#         c_quants = {}

#         for deriv in del_eval_dic:
#             key = '_'.join(deriv.split('_')[1:-1])
#             c_key = None

#             if conv_cos is not None and key in conv_cos:
#                 c_deriv, c_key, c_val = edh.convert_to_cos_derivative(del_eval_dic[deriv],key,params_dic[key])
#             elif conv_log is not None and key in conv_log:
#                 c_deriv, c_key, c_val = edh.convert_to_log_derivative(del_eval_dic[deriv],key,params_dic[key])
#             else:
#                 conv_dic[deriv] = del_eval_dic[deriv]

#             if c_key is not None:
#                 c_quants[key] = (c_key, c_val)

#                 prefix,suffix = deriv.split(key)
#                 n_key = prefix + c_key + suffix
#                 conv_dic[n_key] = c_deriv

#         return conv_dic, c_quants

In [ ]:
# ant_pat_symbs_string = 'f Mc tc ra dec psi gmst0'
# def calc_det_responses_derivs(wf, deriv_symbs_string, f_arr, params_dic, use_rot=1, label='hf', step=1e-9, method='central', order=2, n=1):

#     wf_symbs_list = wf.wf_symbs_string.split(' ')
#     deriv_symbs_list = deriv_symbs_string.split(' ')

#     if 'f' in wf_symbs_list:
#         wf_symbs_list.remove('f')
#     if 'f' in deriv_symbs_list:
#         deriv_symbs_list.remove('f')

    
#     ap_symbs_list = ant_pat_symbs_string.split(' ')
#     if 'f' in ap_symbs_list:
#         ap_symbs_list.remove('f')

#     dr_symbs_list = bfs.reduce_symbols_strings(wf.wf_symbs_string,ant_pat_symbs_string).split(' ')
#     dr_params_list = list(bfs.get_sub_dict(params_dic,dr_symbs_list).values())

#     def dr_func(f_arr,*dr_params_list):
#         wf_list = []
#         for i,el in enumerate(wf_symbs_list):
#             wf_list.append(dr_params_list[dr_symbs_list.index(el)])

#         ap_list = []
#         for i,el in enumerate(ap_symbs_list):
#             ap_list.append(dr_params_list[dr_symbs_list.index(el)])

#         hfp, hfc = wf.eval_np_func(f_arr, wf_list)
#         Fp, Fc, Flp = 1, 0, 1

#         return Flp * (hfp * Fp + hfc * Fc)

#     return wfd_num.part_deriv_hf_func(dr_func, dr_symbs_list, deriv_symbs_list, f_arr, params_dic, pl_cr=0, compl=1, label=label, step=step, method=method, order=order, n=n)

In [ ]:
# def calc_det_responses(self, wf, inj_params):
#         hfp, hfc = wf.eval_np_func(self.f,bfs.get_sub_dict(inj_params,wf.wf_symbs_string))
#         return hfp

# def calc_det_responses_derivs_num(self, deriv_variables, inj_params, wf, deriv_symbs_string, conv_cos, conv_log, use_rot, step=1e-9, method='central', order=2, n=1):
    
#     self.hfp = calc_det_responses(net, net.wf, inj_params)
#     del_hf = calc_det_responses_derivs(wf,deriv_symbs_string,self.f,inj_params,use_rot,'hf',step,method,order,n)
#     del_hf, c_quants = get_conv_del_eval_dic(del_hf, inj_params, conv_cos, conv_log, deriv_symbs_string)
    
#     return del_hf

In [ ]:
# del_hf = calc_det_responses_derivs_num(net, net.deriv_variables, inj_params, net.wf, deriv_symbs_string, conv_cos, conv_log, use_rot)

In [ ]:
# def calc_fisher_cov_matrices(self, del_hf, only_net, psd, df, cond_sup):
    
#     del_hf_sub_dict = bfs.get_sub_dict(del_hf,('hf',),0)
#     if not only_net:
#         self.fisher, self.cov, self.wc_fisher, self.cond_num = fat.calc_fisher_cov_matrices(list(del_hf_sub_dict.values()), psd,self.f,0,df,cond_sup)
#         return self.fisher
#     else:
#         fisher,_,_,_ = fat.calc_fisher_cov_matrices(list(del_hf_sub_dict.values()),psd,self.f,1,df,cond_sup)
#         return fisher

In [ ]:
# calc_fisher_cov_matrices(net, del_hf, 0, psd1, None, cond_sup)

In [ ]:
# net.wc_fisher, net.cond_num = fat.check_well_conditioned(net.fisher,cond_sup)

In [ ]:
# net.cov = fat.calc_cov_from_fisher(net.fisher,net.wc_fisher) 
# net.inv_err = fat.inv_err_from_fisher_cov(net.fisher,net.cov, by_element)
# net.errs = fat.get_errs_from_cov(net.cov,net.deriv_variables)

In [ ]:
# net.errs

In [ ]:
# net.cov

In [ ]:
# def calc_snrs(self, hf, psd, only_net, df):
#     snr,snr_sq = snr_mod.snr_snr_sq_freq_array(hf, psd, self.f, df)
#     if not only_net:
#         self.snr = snr
#         self.snr_sq = snr_sq
#     return snr

In [ ]:
# wf = net.wf
# hfp, hfc = wf.eval_np_func(net.f,bfs.get_sub_dict(net.inj_params,wf.wf_symbs_string))
# hf = hfp

In [ ]:
# calc_snrs(net, hf, psd1, 0, None)